In [3]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
import logging
logging.basicConfig(level=logging.INFO)

# local imports
from src.fetch.google import fetch_google_reviews, fetch_place_id, fetch_restaurant_info
from src.nlp.dish_extractor import extract_dishes, _cached_extract_single
from src.ranking.functions import assign_rankings

# Clear dish extractor cache to ensure fresh model load
_cached_extract_single.cache_clear()


In [4]:
class Restaurant():
    def __init__(self, place_id):
        self.id = place_id
        
        #fetching restaurant info
        info = fetch_restaurant_info(place_id)
        self.name = info.get("name")
        self.address = info.get("address")
        self.id_check = info.get("id")
        
        #check if fetch worked
        if str(place_id).strip() != str(self.id_check).strip():
            raise ValueError(f"Restaurant ID mismatch: {place_id} vs {self.id_check}")
        
        #putting in placeholders for important later info
        self.reviews = []
        self.google_url = ""
        self.homepage_url = ""
        self.menu_url = ""

In [5]:
r1_id = fetch_place_id("funkyfish berlin")
r2_id = fetch_place_id("standard pizza west berlin")
r3_id = fetch_place_id("borschert berlin")
r4_id = fetch_place_id("Gio's Berlin")

r4_id 

INFO:src.fetch.google:
 Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ID = ChIJU7-KvONQqEcRCmPCCvo-Fg4

INFO:src.fetch.google:
 Found restaurant: Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ID = ChIJc2uwDI1RqEcRfAsy3YxyfIY

INFO:src.fetch.google:
 Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ID = ChIJVfEmINtRqEcRsch0pslaanQ

INFO:src.fetch.google:
 Found restaurant: Gio's
Dresdener Str. 16, 10999 Berlin, Deutschland
ID = ChIJ65Liv5JPqEcRodr8aFFNAvE



'ChIJ65Liv5JPqEcRodr8aFFNAvE'

In [6]:
r1 = Restaurant(r1_id)
r4 = Restaurant(r4_id)

In [7]:
r1_id

'ChIJU7-KvONQqEcRCmPCCvo-Fg4'

In [8]:
r1_reviews = fetch_google_reviews(r1_id)
r2_reviews = fetch_google_reviews(r2_id)
r3_reviews = fetch_google_reviews(r3_id)
r4_reviews = fetch_google_reviews(r4_id)

INFO:src.fetch.google:✅ Retrieved 5 reviews for FunkyFisch (EN)
INFO:src.fetch.google:✅ Retrieved 5 reviews for Standard Pizza West (EN)
INFO:src.fetch.google:✅ Retrieved 5 reviews for Borchardt (EN)
INFO:src.fetch.google:✅ Retrieved 5 reviews for Gio's (EN)


In [7]:
r4_reviews

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Steven Tice',
  'text': "First experience with Georgian food, and it was amazing! The service was fantastic, and our server was kind enoug

In [9]:
extract_dishes(r4_reviews, True)

INFO:src.nlp.dish_extractor:Loading model: google/flan-t5-large ...
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:src.nlp.dish_extractor:Model loaded successfully.
INFO:src.nlp.dish_extractor:📝 Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.
INFO:src.nlp.dish_extractor:🍽️ mini georgian bread, dessert, adjapsandali

INFO:src.nlp.dish_extractor:📝 What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.
INFO:src.nlp.dish_extractor:🍽️ none

INFO:src.nlp.dish_extractor:📝 First experience with Georgian food, and it was amazing! The service was fantastic, and our server was kind enough to help us pick out excellent options for our first Georgian food experience (especially because we

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [{'name': 'mini georgian bread', 'ranking': None},
   {'name': 'dessert', 'ranking': None},
   {'name': 'adjapsandali', 'ranking': None}],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Ste

In [9]:
assign_rankings(r4_reviews, True)

Assigned ranking -1 to dish 'dessert' in review #None
Assigned ranking -1 to dish 'adjapsandali' in review #None
Assigned ranking -1 to dish 'mini georgian bread' in review #None
Assigned ranking -1 to dish 'dumplings' in review #None
Assigned ranking -1 to dish 'khachapuri imeruli' in review #None
Assigned ranking -1 to dish 'khinkali with cheese' in review #None
Assigned ranking -1 to dish 'khinkali with meat' in review #None


In [10]:
r4_reviews

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [{'name': 'dessert', 'ranking': -1},
   {'name': 'adjapsandali', 'ranking': -1},
   {'name': 'mini georgian bread', 'ranking': -1}],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Steven Ti

In [ ]:
def flatten_and_sort_dishes(reviews):
    """
    Extracts all dishes from a list of reviews and returns
    them as a flat, sorted list of dicts with metadata.
    """
    dish_entries = []

    for review in reviews:
        source = review.get("source")
        author = review.get("author")
        url = review.get("url")
        date = review.get("date")

        for dish in review.get("dishes", []):
            dish_entries.append({
                "name": dish.get("name"),
                "ranking": dish.get("ranking"),
                "author": author,
                "url": url,
                "date": date,
                "source": source
            })

    # Sort by ranking (None values last)
    dish_entries.sort(key=lambda d: (d["ranking"] is None, d["ranking"]))
    return dish_entries

flatten_and_sort_dishes(r4_reviews)

[{'name': 'adjapsand',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'date': 1751374444,
  'source': 'google'},
 {'name': 'adapsandali',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'date': 1751374444,
  'source': 'google'},
 {'name': 'mini georgian bread',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'date': 1751374444,
  'source': 'google'},
 {'name': 'none',
  'ranking': -1,
  'author': 'Gaia Tassinari',
  'url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'date': 1755459365,
  'source': 'google'},
 {'name': 'none',
  'ranking': -1,
  'author': 'Steven Tice',
  'url': 'https://www.google.com/maps/contrib/102575073644000772513/reviews',
  'date': 1757104922,
  'source': 'google'},
 {'name': 'dumplings',
  'rank